In [ ]:
import pandas as pd
import googlemaps
import folium

def geocode_and_create_map(input_file_path, output_file_path, address_column='Indirizzo', map_output_path='map.html'):
    """
    Geocodes addresses in a DataFrame using Google Maps API and creates a map with markers.
    
    Args:
        input_file_path (str): Path to the input CSV file.
        output_file_path (str): Path to save the output CSV file with geocoded results.
        address_column (str): Column name containing the addresses to geocode. Default is 'Indirizzo'.
        map_output_path (str): Path to save the generated HTML map.
        
    Returns:
        None: Saves geocoded CSV and an interactive map as an HTML file.
    """
    # Initialize Google Maps client with your API key
    gmaps = googlemaps.Client(key="YOUR_API_KEY")  # Replace with your API key
    
    # Load the data
    df = pd.read_csv(input_file_path)
    
    # Prepare storage for latitude and longitude
    latitudes = []
    longitudes = []
    
    # Geocode each address
    for address in df[address_column]:
        try:
            geocode_result = gmaps.geocode(address)
            if geocode_result:
                # Extract the first result's geometry location
                location = geocode_result[0]['geometry']['location']
                latitudes.append(location['lat'])
                longitudes.append(location['lng'])
            else:
                latitudes.append(None)
                longitudes.append(None)
        except Exception as e:
            print(f"Error geocoding address {address}: {e}")
            latitudes.append(None)
            longitudes.append(None)
    
    # Add the results to the DataFrame
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes
    
    # Save the updated DataFrame
    df.to_csv(output_file_path, index=False)
    
    # Create the map
    map_center = [df['Latitude'].mean(), df['Longitude'].mean()]  # Center the map around the average location
    m = folium.Map(location=map_center, zoom_start=12)
    
    # Add markers for each geocoded location
    for _, row in df.iterrows():
        if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=row[address_column],
            ).add_to(m)
    
    # Save the map to an HTML file
    m.save(map_output_path)
    print(f"Map with markers saved to {map_output_path}")

# Example usage
input_path = '/mnt/data/Consultori Mappatura Cerca.ipynb'  # Replace with your actual file path
output_path = '/mnt/data/Consultori_Mappatura_Geocoded.csv'
map_output_path = '/mnt/data/Consultori_Map.html'
geocode_and_create_map(input_path, output_path, address_column='Indirizzo', map_output_path=map_output_path)
